<a href="https://colab.research.google.com/github/hhnnhh/berlin_datascience_lab/blob/master/exploration/01_EDA/EDA_important_car_features_biased_stations_software.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exploratory data analysis:
by Hannah Bohle

Explored features are: 
- engine size, engine size orig
- model
- station num, analyzer num
- cyl, trans type, dual exhaust
- software version

Summary of my suggestions following the EDA:
I'd suggest to use ENGINE_SIZE; CYL and eventually TRANS_TYPE.

* there seems an odd station with 50% fail?!
* Same problem with Software_Version, even if the bias is not that obvious

SUMMARY:

1. use **ENGINE_SIZE** but *not* **ENGINE_SIZE_ORIG**
2. use **MAKE** but *not* **MODEL** ('MODEL' needs to be cleaned, but 'MAKE' will eventually contain similar information. Consider using MODEL if MAKE is very good)
3. **STATION_NUM**: not an interesting variable for our algorithm, because it wont explain the emissions. However, I didn't manage to calculate the percentage of PASS/FAIL per station, which would be interesting to report to the client, there seems to be one biased station with 50% fails?!
4. I'd suggest to use **CYL** after getting rid of the letter 'R'. Can and should be converted to float? 
5. **TRANS_TYPE** (automatic/manual) - I read an article that automatic cars can have lower emissions (but only certain automatic cars) --> take or leave?
6. **DUAL_EXHAUST** (yes/no), binary variables.  DE is very imbalanced with only 0.6% YES. Not so interesting I guess.
7. **Software Version**: there are SV that seem to have more PASSES than FAILS? If this is really a bias, then I guess we should report it to the client. *--> LOOK INTO IT.*

I *have not* looked into Abort_Codes, Abort_Code_Description (because I think Isabelle has already done it) and ANALYZER_NUM

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
%matplotlib inline

In [2]:
from google.colab import drive

drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## Loading data frame

In [ ]:
#data path:
data_path = "/content/drive/My Drive/WIMLDS/sample.csv"
#data load
df = pd.read_csv(data_path, lineterminator='\n')

## Preparing the data for exploration



    engine size, engine size orig
    model
    station num, analyzer num
    cyl, trans type, dual exhaust
    software version
    abort codes, abort code description


In [ ]:
keep_cols = ['VIR_CERT_NUMBER', 'STATION_NUM', 'ANALYZER_NUM', 'INSPECTOR_NUM', 'TEST_SDATE', 'TEST_EDATE', 'VIN', 'VIN_SOURCE', 'VEH_LICENSE', 'GVW_TYPE', 'GVWR', 'MODEL_YEAR', 
             'MAKE', 'MODEL', 'CYL', 'ENGINE_SIZE', 'TRANS_TYPE', 'DUAL_EXHAUST', 'ODOMETER', 'FUEL_TYPE', 'VEHICLE_TYPE', 'ZIP_CODE', 'CERTIFICATION', 'ESC', 'VRT_RECORD_ID', 
             'ABORT_CODE', 'SOFTWARE_VERSION', 'INSP_COMMENTS', 'OVERALL_RESULT', 'ABORT_CODE_OTHER_DESC', 'MECHANIC_LAST_NAME', 'MECHANIC_FIRST_NAME', 'MECHANIC_MIDDLE_INIT', 
             'ENGINE_SIZE_ORIG', 'VID_TEST_TYPE']

df = df[keep_cols]

In [ ]:
df.groupby('OVERALL_RESULT').size().plot.bar()

In [ ]:
#reducing df by overall-result = P,F
df = df[df.OVERALL_RESULT.isin(['P', 'F'])]

In [ ]:
df.head()

##1. (A) ENGINE SIZE and (B) ENGINE SIZE ORIG

My suggestion: use ENGINE_SIZE but *not* ENGINE_SIZE_ORIG!

1. what do they encode?
*   (A) Engine size: Engine Size in cubic centimeters, cubic inches or liters , 4 char
*   (B) Engine size orig: Engine Size Other manually entered by tech., 10 Char.

--> (A) is continous, while (B) is a string (eg. 3.5L, 4.0L, ..)

2. are they related?
 --> haven't found a relation or pattern, but it is difficult to determine because (B) has loads of different strings

3. are they correlated with the overall results?
--> the boxplot does not show an obvious difference
--> Failing cars seem to have a slightly smaller engine size but it might be confounded with age, as older cars will have smaller engines and will fail more often

*Open task:* We could convert (B) to float by getting rid of the letters ('L') but also some other letters ('I') - but we would have to look closer into the variable, I doubt that it's worth it.



## (A) ENGINE SIZE

In [ ]:
#ENGINE SIZE DOES NOT SEEM TO HAVE AN IMPACT ON RESULT
# Failing cars seem to have a slightly smaller engine size but it might be confounded with age, as older cars will have smaller engines and fail more often
fig, ax = plt.subplots(figsize=(10,8))
sns.boxplot(data=df, x='OVERALL_RESULT', y='ENGINE_SIZE')

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
df.groupby('ENGINE_SIZE').size().sort_values(ascending=False).head(100).plot.bar()

In [ ]:
fig, ax = plt.subplots(figsize=[20,8])
sns.countplot(df['ENGINE_SIZE'])
#sns.countplot((df['ENGINE_SIZE']>2000) & (df['ENGINE_SIZE']<4000))

In [ ]:
sns.distplot(df["ENGINE_SIZE"])

##(B) ENGINE SIZE ORIG

In [ ]:
#Engine size is a numerical and continous variable
#Engine size Orig = object --> several letter characters such as L
df.ENGINE_SIZE_ORIG.describe()

In [ ]:
df.ENGINE_SIZE_ORIG.value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
df.groupby('ENGINE_SIZE_ORIG').size().sort_values(ascending=False).head(100).plot.bar()

#MODEL

My preliminary suggestion: use the variable MAKE before using MODEl. MAKE is cleaner, is less work and might contain similar information. **If MAKE is improving a lot, then we might think about cleaning MODEL and use it as well.**

We could think about:
1. using fuzzywuzzy to compare strings for similarity
2. use a clustering algorithm to group the models?
3. strip either all numbers or at least 4WD, 2WD and AWD to reduce number of cars
4. unique cars - do we need them? are they caused by mistakes?


In [ ]:
df.MODEL.describe()

In [ ]:
df.MODEL.unique()

In [ ]:
df.MODEL.value_counts()

In [ ]:
len(df.MODEL.unique()) # es gibt nur 2740 unique models von 235177?

In [ ]:
len(df.MODEL)

In [ ]:
df['AGE'] = pd.to_datetime(df["TEST_EDATE"]).dt.year - df["MODEL_YEAR"]

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
df.groupby('MODEL').size().sort_values(ascending=False).head(100).plot.bar()

In [ ]:
md = df.groupby('MODEL')#.size().sort_values(ascending=False).head(100)
size = md.size()
size[size > 100].index

In [ ]:
frequent_models = df[df.MODEL.isin(size[size >= 500].index)]

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
pd.DataFrame(frequent_models.groupby(['MODEL', 'OVERALL_RESULT']).size()).unstack().plot.bar(ax=ax)
#pd.DataFrame(df.groupby(['MODEL', 'OVERALL_RESULT']).size().sort_values(ascending=False).head(100)

In [ ]:
# some of the cars DO seem to have a higher FAIL ratio (--> F150 Pickup, Ranger), but there are other Pickups that don't
fig, ax = plt.subplots(figsize=(20,6))
pd.DataFrame(frequent_models.groupby(['MODEL', 'OVERALL_RESULT']).size()/frequent_models.groupby('MODEL').size()).unstack().plot.bar(ax=ax, stacked=True)

In [ ]:
#as expected, older cars seem to fail more often. There is no obvious relation between MODEL and RESULT.
fig, ax = plt.subplots(figsize=(20,6))
plt.xticks(rotation=90)
sns.scatterplot(data=frequent_models, x='MODEL', y='AGE', hue='OVERALL_RESULT', s=100, alpha=.05)

In [ ]:
#some of the models have more wrong odometer values (but here, a ration would be more informative than absolute values..)
fig, ax = plt.subplots(figsize=(20,6))
df[(df.ODOMETER == 88888888) | (df.ODOMETER == 100000)| (df.ODOMETER == 8888888)| (df.ODOMETER == 88888) | (df.ODOMETER == 0)].groupby('MODEL').size().sort_values(ascending=False).head(100).plot.bar()

#STATION_NUM 
Some of the stations do loads of tests, but no unexpected patterns in this variable by now. 
Failed to plot percentages of overall_result Pass/Fail by station, which would be nice to have but not mandatory I guess (see below).


In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
df.groupby('STATION_NUM').size().sort_values(ascending=False).head(100).plot.bar()

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
pd.DataFrame(df.groupby(['STATION_NUM', 'OVERALL_RESULT']).size()/df.groupby('STATION_NUM').size()).unstack().plot.bar(ax=ax, stacked=True)

### calculate percentage of fails and passes per station to find out whether there are stations which are biased. 
--> I don't think that this will be important for any model but might help the client or might help us to understand the data better and/or remove data from biased stations?

*percentage calculation by Isabelle Nguyen*

In [ ]:
top_indices = df.groupby(['STATION_NUM']).size().sort_values(ascending=False).head(20).index#.head(240).unstack()

In [ ]:
fig, ax = plt.subplots(figsize=(14,6))
top_stations = df[df.STATION_NUM.isin(top_indices)]
pd.DataFrame(top_stations.groupby(['STATION_NUM','OVERALL_RESULT']).size()/top_stations.groupby('STATION_NUM').size()).unstack().fillna(0).plot.bar(ax=ax, stacked=True)
plt.show()

In [ ]:
df[df.STATION_NUM == 'VP001'].groupby('OVERALL_RESULT').size().plot.bar()

In [ ]:
#df['OVERALL_RESULT'].value_counts(normalize=True) * 100
tab = pd.crosstab(df['OVERALL_RESULT'],df['STATION_NUM']).apply(lambda r: r/r.sum(), axis=1)
tab

In [ ]:
df = df[df.OVERALL_RESULT.isin(['P', 'F'])]


In [ ]:
#df.groupby(['STATION_NUM', 'OVERALL_RESULT']).size().reset_index(name='counts')
counts = df.groupby(['STATION_NUM', 'OVERALL_RESULT']).size().reset_index(name='counts')

In [ ]:
counts

cyl, trans type, dual exhaust

#CYLINDER
has a strange value "R" - but it is not frequent (N=90) and can eventually be removed (?)
tranforming to float makes the variable more consistent

In [ ]:
df.CYL.describe()




In [ ]:
df.CYL.value_counts()

In [ ]:
df = df[df.CYL != 'R']

In [ ]:
df["CYL"] = df.CYL.astype(float)
df.CYL.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
df.groupby('CYL').size().sort_values(ascending=False).head(100).plot.bar()

#trans type = transmission type
binary condition with categories "A" & "M"
which apparently refer to "automatic" or "manual" transmission
Percentages: 
* A    83% automatic
* M    16% manual

--> I don't think it will be a useful feature


In [ ]:
df.TRANS_TYPE.describe()

In [ ]:
df.TRANS_TYPE.value_counts()

In [ ]:
df['TRANS_TYPE'].value_counts(normalize=True) * 100

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
df.groupby('TRANS_TYPE').size().sort_values(ascending=False).head(100).plot.bar()

#dual exhaust
might make a car more efficient (https://resource-center.meineke.com/dual-exhausts-just-for-looks/)

I'd suggest not to use dual_exhaust further.
The variable is binary (Yes/No) and has only a very few occurrences of Yes (0.6%). It encodes if the car has two exhaust pipes, which makes bigger cars more efficient.

Percentage of Dual Exhaust Cars: 

* NO   99.4%
* YES   0.6%




In [ ]:
df.DUAL_EXHAUST.describe()

In [ ]:
df.DUAL_EXHAUST.value_counts()

In [ ]:
df['DUAL_EXHAUST'].value_counts(normalize=True) * 100

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
df.groupby('DUAL_EXHAUST').size().sort_values(ascending=False).head(100).plot.bar()

# SOFTWARE VERSION
the numbers are strange - the version numbers all have three digits, but there is only one frequently used software version 5xx, but mostly 1xx and 2xx

In [ ]:
df.SOFTWARE_VERSION.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
df.groupby('SOFTWARE_VERSION').size().sort_values(ascending=False).plot.bar()

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
pd.DataFrame(frequent_models.groupby(['SOFTWARE_VERSION', 'OVERALL_RESULT']).size()).unstack().plot.bar(ax=ax)

In [ ]:
df.groupby('SOFTWARE_VERSION')['OVERALL_RESULT'].count()